# Info

In this notebook we use the Lennard-Jones toysystem we created in `lennard_jones_toysystem.ipynb` to play around a bit and learn how to create a `CustomExternalForce`. 

## Targets
* create a "droplet" of Lennard-Jones particles
* run a normal simulation
* add a spherical constraint using `CustomExternalForce` and run a restraint simulation

# Intialization

In [ ]:
from simtk import openmm as mm
from simtk.openmm import app
from simtk.unit import *
import numpy as np

import mdtraj
import nglview

import matplotlib.pyplot as plt
%matplotlib inline

# functions to setup the simulation

We define different functions to create the simulation. This way we can focus on the business logic later on and do not have a overwhelming part of repeating implementation code.

**Functions:**
* `initialize_system(n_particles, mass, box_size)` <br>
    Function to initialize a Lennard Jones system.
* `create_NonBonndedForce(n_particles, sigma, epsilon, charge)` <br>
    Add a NonbondedForce to the system.
* `create_topology(n_particles, box_size)` <br>
    Function to create the toplogy of the Lennard Jones system.

In [ ]:
def initialize_system(n_particles, mass, box_size):
    """
    Function to initialize a Lennard Jones system.
    
    Parameters
    ----------
    n_particles : int
        Number of particles in the system
    mass : Quantity
        mass of particle
    box_size : Quantity
        Box length
        
    Returns
    -------
    system : simtk.openmm.openmm.System
    """
    
    box_vectors = np.diag([box_size/angstrom for i in range(3)])*angstrom

    # Create a system and add particles to it
    system = mm.System()
    for index in range(n_particles):
        # Particles are added one at a time
        # Their indices in the System will correspond with their indices in the Force objects we will add later
        system.addParticle(mass)
        
    box_vectors = np.diag([box_size for i in range(3)])
    system.setDefaultPeriodicBoxVectors(*box_vectors)

    return system

In [ ]:
def create_NonBonndedForce(n_particles, sigma, epsilon, charge):
    """
    Add a NonbondedForce to the system.
    
    Parameters
    ----------
    n_particles : int
        Number of particles in the system.
    sigma : float, optional
        Sigma of the Lennard-Jones potential.
    epsilon : float, optional
        Epsilon of the Lennard-Jones potential. Default is `0`.
    charge : float, optional
        Charge of the particles. Default is `0`.
    Returns
    -------
    force : simtk.openmm.openmm.NonbondedForce
        NonbondedForce assigned to all particles
    """
    # Add Lennard-Jones interactions using a NonbondedForce
    force = mm.NonbondedForce()
    force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)

    # all particles must have parameters assigned for the NonbondedForce
    for index in range(n_particles): 
        # Particles are assigned properties in the same order as they appear in the System object
        force.addParticle(charge, sigma, epsilon)
    
    force.setCutoffDistance(3.0 * sigma) # set cutoff (truncation) distance at 3*sigma
    force.setUseSwitchingFunction(True) # use a smooth switching function to avoid force discontinuities at cutoff
    force.setSwitchingDistance(2.5 * sigma) # turn on switch at 2.5*sigma
    force.setUseDispersionCorrection(True) # use long-range isotropic dispersion correction
  
    return force

In [ ]:
def create_topology(n_particles, box_size):
    """
    Function to create the toplogy of the Lennard Jones system.
    
    Parameters
    ----------
    n_particles : int
        Number of particles in the system
    
    Returns
    -------
    top : simtk.openmm.app.topology.Topology
        Topology for the system
    """
    top = app.Topology()
    chain = top.addChain()
    for i in range(n_particles):
        residue = top.addResidue(name='Ar', chain=chain, id=i)
        top.addAtom('Ar',element=app.Element.getBySymbol('Ar') , residue=residue)
        
    box_vectors = np.diag([box_size/angstrom for i in range(3)])*angstrom
    top.setPeriodicBoxVectors(box_vectors)
    
    return top

## Visualization

Here are just some functions to visualize our system later on. You can skip this part.

* `boxvectors2length(box_vectors)` <br>
    Converts box_vectors to lengths and angles.
* `viz_traj(traj)` <br>
    Creates a nglview object

In [ ]:
def boxvectors2length(box_vectors):
    """
    Converts box_vectors to lengths and angles.
    
    Parameters
    ----------
    box_vectors : Quantity
        Box vectors
    
    Returns
    -------
    box_length : Quantity
        Box length (a, b, c)
    box_angles : Quantity
        Box angles (alpha, beta, gamma)
    """
    ((lx, _ , _  ),
     (xy, ly, _  ), 
     (xz, yz, lz))   = ( box_vectors )
    
    a = lx
    b = sqrt(ly**2 + xy**2)
    c = sqrt(lz**2 + xz**2 + yz**2)
    alpha = acos(( xy*xz + ly*yz ) / (b * c)).in_units_of(degree)
    beta  = acos(xz / c).in_units_of(degree)
    gamma = acos(yz / b).in_units_of(degree)
    return Quantity((a, b, c)), Quantity((alpha, beta, gamma))

In [ ]:
def viz_traj(traj):
    "Creates a nglview object."
    view = nglview.show_mdtraj(traj, use_box=True) # gui=True for more options

    view.add_spacefill('all')
    view.add_unitcell()
    # update camera type
    view.camera = 'orthographic'

    view.center()
    return view

# Input

First of all, we have to define our input again.

> Note: This time we also define `space` as an variable for how much space should be around our "droplet".

In [ ]:
# Force field
mass = 39.9 * amu
charge = 0.0 * elementary_charge
sigma = 3.4 * angstroms
epsilon = 0.238 * kilocalories_per_mole

# System
n_particles = 256
box_size = 75 * angstrom
space = 10 * angstrom # space around the system

# MD settings
timestep = 1.0 * femtoseconds

## generate positions

We have to define the positions of our atoms. To create a droplet we go the easy way to only define random positions in a subset of our box.

In [ ]:
positions = (box_size - space*2) * np.random.rand(n_particles, 3)  + space

# Normal Simulation

Let's run a normal simulation as reference to see what's going on.

In [ ]:
# define a system
system = initialize_system(n_particles, mass, box_size)

# create a NonbondedForce (with charge=0 -> only Lennard-Jones interactions)
force = create_NonBonndedForce(n_particles, sigma, epsilon, charge)
force_index = system.addForce(force) 

# define integrator
integrator = mm.VerletIntegrator(timestep)

# create topology
topology = create_topology(n_particles, box_size)

# define a simulation context
simulation = app.Simulation(topology=topology, system=system, integrator=integrator)

Set the starting positions

In [ ]:
simulation.context.setPositions(positions)

Visualize the system.

In [ ]:
mdtraj_topology = mdtraj.Topology.from_openmm(simulation.topology)
unitcell_lengths, unitcell_angles = boxvectors2length(simulation.topology.getPeriodicBoxVectors())
traj = mdtraj.Trajectory(positions/nanometers, mdtraj_topology,
                         unitcell_lengths=unitcell_lengths.value_in_unit(nanometer),
                         unitcell_angles=unitcell_angles.value_in_unit(degree))

viz_traj(traj)

As you can see, we defined atoms in a inner part of the box with a spacer around them.

## Minimize

We need to minimize the system to avoid overlapping atoms.

In [ ]:
simulation.minimizeEnergy()

Visualization.

In [ ]:
tmp_positions = simulation.context.getState(getPositions=True).getPositions()
unitcell_lengths, unitcell_angles = boxvectors2length(simulation.topology.getPeriodicBoxVectors())
traj = mdtraj.Trajectory(tmp_positions/nanometers, mdtraj_topology,
                        unitcell_lengths=unitcell_lengths.value_in_unit(nanometer),
                        unitcell_angles=unitcell_angles.value_in_unit(degree))

viz_traj(traj)

## simulation

Let's run the simulation.

We first add `reporters`, then create velocities for a given `temperature` and run the simulation for a few steps.

In [ ]:
#add reporters
simulation.reporters.append(app.DCDReporter('trajectory.droplet.normal.dcd', 100))
simulation.reporters.append(app.StateDataReporter('thermo.droplet.normal.csv', 100,
                                                  step=True,
                                                  potentialEnergy=True,
                                                  kineticEnergy=True,
                                                  totalEnergy=True,
                                                  temperature=True))

In [ ]:
# set velocities
simulation.context.setVelocitiesToTemperature(300 * kelvin)

In [ ]:
simulation.step(10000)

## visualize

Let's have a look what happened.

In [ ]:
mdtraj_topology = mdtraj.Topology.from_openmm(simulation.topology)
traj = mdtraj.load_dcd('trajectory.droplet.normal.dcd', mdtraj_topology)
view = viz_traj(traj)
view

As expected argon behaves like a gas at 300 Kelvin and spread across the box.

# Spherical potential

Let's add a spherical potential.

First we repeat the steps from the previous example.

In [ ]:
# define a system
system = initialize_system(n_particles, mass, box_size)

# create a NonbondedForce (with charge=0 -> only Lennard-Jones interactions)
force = create_NonBonndedForce(n_particles, sigma, epsilon, charge)
force_index = system.addForce(force) 

# define integrator
integrator = mm.VerletIntegrator(timestep)

# create topology
topology = create_topology(n_particles, box_size)

## Add a spherical constraint

Now we want to add an extra `force` to our system to prevent this.

We can define a force following a custom equation by using `openmm.CustomExternalForce(equation)`.
`equation` is just a plain expression of the energy.

It may depend the on `x`, `y` and `z` coordinate and any other arbitrary parameter defined. These parameter can be added with `addPerParticleParameter()`and `addGlobalParameter()`. Variables can be placed in separate expressions using `;` to end the previous one.

Expressions may involve the operators `+` (add), `-` (subtract), `*` (multiply), `/` (divide), and `^` (power), and the following functions: `sqrt`, `exp`, `log`, `sin`, `cos`, `sec`, `csc`, `tan`, `cot`, `asin`, `acos`, `atan`, `sinh`, `cosh`, `tanh`, `erf`, `erfc`, `step`. All trigonometric functions are defined in `radians`, and `log` is the natural logarithm. `step(x) = 0` if `x` is less than `0`, `1` otherwise.

In [ ]:
force = mm.CustomExternalForce(
    '10*max(0, r-{r0})^2; r=sqrt((x-{x0})^2+(y-{x0})^2+(z-{z0})^2)'.format(
        r0=2.0,
        x0=box_size/2.0/nanometer,
        y0=box_size/2.0/nanometer,
        z0=box_size/2.0/nanometer
    )
)
for i in range(system.getNumParticles()):
    force.addParticle(i, ())
system.addForce(force)

If one does not want to define all parameters in advance,
one can just implement parameters as variables.

* for per particle parameter with `force.addPerParticleParameter()`.
* for global parameters use `force.addGlobalParameter()`.

```python
force = mm.CustomExternalForce('K*max(0, r-r0)^2; r=sqrt((x-x0)^2+(y-x0)^2+(z-z0)^2)')
# define extra variables
force.addPerParticleParameter('K')
force.addPerParticleParameter('r0')
force.addPerParticleParameter('x0')
force.addPerParticleParameter('y0')
force.addPerParticleParameter('z0')

for i in range(system.getNumParticles()):
    # now use the parameters with units in the order as defined previously (K, r0, x0, y0, z0)
    force.addParticle(i, (10*kilojoule_per_mole, 2*nanometer, box_size/2.0, box_size/2.0, box_size/2.0))
system.addForce(force)
```

## Continue as in normal ...

In [ ]:
# define a simulation context
simulation = app.Simulation(topology=topology, system=system, integrator=integrator)

In [ ]:
# set positions
simulation.context.setPositions(positions)
# set velocities
simulation.context.setVelocitiesToTemperature(300 * kelvin)

In [ ]:
# minimize
simulation.minimizeEnergy()

In [ ]:
# add reporters
simulation.reporters.append(app.DCDReporter('trajectory.droplet.spherical.dcd', 100))
simulation.reporters.append(app.StateDataReporter('thermo.droplet.spherical.csv', 100,
                                                  step=True,
                                                  potentialEnergy=True,
                                                  kineticEnergy=True,
                                                  totalEnergy=True,
                                                  temperature=True))

In [ ]:
simulation.step(10000)

## Visualize

Let's have a look into what we created.

In [ ]:
mdtraj_topology = mdtraj.Topology.from_openmm(simulation.topology)
traj = mdtraj.load_dcd('trajectory.droplet.spherical.dcd', mdtraj_topology)
view = viz_traj(traj)
view